In [5]:
import networkx as nx
import collections
import matplotlib.pyplot as plt
import numpy as np
import random
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import math
from scipy.sparse.linalg import eigsh

def gen_graph(cur_n, g_type,seed=None):
    random.seed(seed)
    if g_type == 'erdos_renyi':
        #print(random.uniform(0.15,0.20))
        g = nx.erdos_renyi_graph(n=cur_n, p=random.uniform(0.15,0.20),seed = seed)
    elif g_type == 'powerlaw':
        g = nx.powerlaw_cluster_graph(n=cur_n, m=random.randint(2,4), p=random.uniform(0.1,0.5),seed = seed)
    elif g_type == 'small-world':
        #print(random.randint(2,5),random.uniform(0.1,0.2))
        g = nx.newman_watts_strogatz_graph(n=cur_n, k=random.randint(2,5), p=random.uniform(0.1,0.2),seed = seed)
    elif g_type == 'barabasi_albert':
        #print(random.randint(2,5))
        g = nx.barabasi_albert_graph(n=cur_n, m=random.randint(2,5),seed = seed)
    elif g_type == 'geometric':
        g = nx.random_geometric_graph(cur_n, random.uniform(0.1,0.4),seed = seed)
    return g
    
    
def gen_new_graphs(graph_type,seed = None):
    random.seed(seed)
    np.random.seed(seed)
    a = np.random.choice(graph_type) if len(graph_type) !=1 else graph_type[0]
    number_nodes = 30
    graph = gen_graph(number_nodes, a,seed)
    #graph =add_super_node(graph)
    active = 1
    nx.set_node_attributes(graph,active, "active")
    return graph   
def input_graph(graph_path,file):
    fh = open(graph_path+str(file)+".txt", "rb")
    GRAPH = nx.read_edgelist(fh)
    fh.close()
    nodes = GRAPH.nodes()
    map = {n:int(i) for i, n in enumerate(nodes)}
    GRAPH = nx.relabel_nodes(GRAPH, map)
    GRAPH.remove_edges_from(nx.selfloop_edges(GRAPH))
    return GRAPH, map


In [23]:
def molloy_reed(g):
  all_degree =   np.array(g.degree())[:,1]
  degs = np.delete(all_degree,-1)
  k = degs.mean()
  k2 = np.mean(degs ** 2)
  if k ==0:
    beta = 0
  else:
    beta = k2/k
  return beta
def global_feature(g): 
    subGraph = g#g.subgraph(np.arange(len(g)-1)) #for supernode
    M = len(subGraph.edges())
    N = len(subGraph)
    degs =   np.array(subGraph.degree())[:,1]
    min_k = np.min(degs)
    
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
    adj_lams = np.sort(eigsh(A, k=N-1, which='LA', return_eigenvectors=False))
    nodes = N
    edges = M
    heterogeneity = div/k1
    density = (2*M)/(N*(N-1))
    resilience = molloy_reed(g)
    power_law_exponent = 1 + N / sum(np.log(degs/min_k))
    modularity = nx_comm.modularity(g,nx_comm.label_propagation_communities(g))
    try:
        varepsilons = nx.algorithms.distance_measures.eccentricity(g).values()
        eccentricity=  np.mean(list(varepsilons))
        diameter = nx.algorithms.distance_measures.diameter(g)
        radius = nx.algorithms.distance_measures.radius(g)
    except: 
        eccentricity = 0
        diameter = 0
        radius = 0
    spectral_radius=adj_lams[-1]
    spectral_gap=  adj_lams[-1]-adj_lams[-2]
    natural_connectivity= np.log2(sum(np.exp(np.real(adj_lams)))/len(adj_lams))
    global_efficiency = nx.algorithms.efficiency_measures.global_efficiency(g)
    assortativity = nx.algorithms.assortativity.degree_assortativity_coefficient(g)
    global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity,power_law_exponent, modularity,eccentricity,diameter,radius,spectral_radius,spectral_gap,natural_connectivity,assortativity))
    return global_properties

# Heterogeneous

In [7]:
graph_path= "../Dataset/HomogeneityGraph/"
file_list =  ['heterogeneous_30','heterogeneous_50','heterogeneous_100','heterogeneous_500']
for i, graph_name in enumerate(file_list):
    GRAPH, _ = input_graph(graph_path,graph_name)
    print(global_feature(GRAPH))
[2.30000000e+01 3.10000000e+01 1.22529644e-01 3.32786885e+00 5.94670407e-01 2.16148170e+00 3.22060354e-01 5.34782609e+00 7.00000000e+00 4.00000000e+00 3.39510706e+00 8.07211753e-01 1.82678582e+00 8.92193309e-03]
[36. 52. 0.08253968 3.32 0.45726496 2.03397491 0.45654586  6.44444444  8. 4. 3.32751135 0.2688529 1.86972708 -0.26708861]
[ 7.60000000e+01  1.18000000e+02  4.14035088e-02  4.25957447e+00  1.14049955e+00  2.04041130e+00  5.17451882e-01  7.89473684e+00  1.00000000e+01  6.00000000e+00  4.43246189e+00  8.70449976e-01  2.16475129e+00 -2.19820003e-01]
[ 4.14000000e+02  6.39000000e+02  7.47447100e-03  4.36726703e+00  1.27767572e+00  2.07480515e+00  5.88949625e-01  1.41908213e+01  1.90000000e+01  1.10000000e+01  4.68377251e+00  3.33456762e-01  2.13131275e+00 -3.01657479e-01]

/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


[2.30000000e+01 3.10000000e+01 1.22529644e-01 3.32786885e+00
 5.94670407e-01 2.16148170e+00 3.22060354e-01 5.34782609e+00
 7.00000000e+00 4.00000000e+00 3.39510706e+00 8.07211753e-01
 1.82678582e+00 8.92193309e-03]
[36.         52.          0.08253968  3.32        0.45726496  2.03397491
  0.45654586  6.44444444  8.          4.          3.32751135  0.2688529
  1.86972708 -0.26708861]
[ 7.60000000e+01  1.18000000e+02  4.14035088e-02  4.25957447e+00
  1.14049955e+00  2.04041130e+00  5.17451882e-01  7.89473684e+00
  1.00000000e+01  6.00000000e+00  4.43246189e+00  8.70449976e-01
  2.16475129e+00 -2.19820003e-01]
[ 4.14000000e+02  6.39000000e+02  7.47447100e-03  4.36726703e+00
  1.27767572e+00  2.07480515e+00  5.88949625e-01  1.41908213e+01
  1.90000000e+01  1.10000000e+01  4.68377251e+00  3.33456762e-01
  2.13131275e+00 -3.01657479e-01]


# Homogeneous

In [13]:
graph_path= "../Dataset/HomogeneityGraph/"
file_list =  ['homogeneous_30','homogeneous_50','homogeneous_100','homogeneous_500']
for i, graph_name in enumerate(file_list):
    GRAPH, _ = input_graph(graph_path,graph_name)
    print(global_feature(GRAPH))
[29. 48. 0.1182266   3.69473684  0.35632184  1.88549419  0.32986111  4.86206897  6. 4. 3.62663654  0.60171806  2.10565352 -0.38157895]
[ 5.00000000e+01  7.80000000e+01  6.36734694e-02  3.64516129e+00  5.08205128e-01  1.96414874e+00  4.16009204e-01  6.46000000e+00  8.00000000e+00  5.00000000e+00  3.61924707e+00  3.16089730e-01  1.98560318e+00 -2.33918809e-01]
[ 9.40000000e+01  1.50000000e+02  3.43170899e-02  3.95973154e+00  7.55177305e-01  1.97689616e+00  5.22444444e-01  8.12765957e+00  1.10000000e+01  6.00000000e+00  4.13464925e+00  3.96779272e-01  2.10133017e+00 -1.19788083e-01]
[ 4.66000000e+02  7.87000000e+02  7.26383313e-03  4.17228226e+00  7.92584433e-01  1.91577842e+00  5.77348882e-01  1.35364807e+01  1.70000000e+01  1.00000000e+01  4.34311025e+00  1.69668607e-01  2.19029031e+00 -2.44196519e-01]

/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


[29.         48.          0.1182266   3.69473684  0.35632184  1.88549419
  0.32986111  4.86206897  6.          4.          3.62663654  0.60171806
  2.10565352 -0.38157895]
[ 5.00000000e+01  7.80000000e+01  6.36734694e-02  3.64516129e+00
  5.08205128e-01  1.96414874e+00  4.16009204e-01  6.46000000e+00
  8.00000000e+00  5.00000000e+00  3.61924707e+00  3.16089730e-01
  1.98560318e+00 -2.33918809e-01]
[ 9.40000000e+01  1.50000000e+02  3.43170899e-02  3.95973154e+00
  7.55177305e-01  1.97689616e+00  5.22444444e-01  8.12765957e+00
  1.10000000e+01  6.00000000e+00  4.13464925e+00  3.96779272e-01
  2.10133017e+00 -1.19788083e-01]
[ 4.66000000e+02  7.87000000e+02  7.26383313e-03  4.17228226e+00
  7.92584433e-01  1.91577842e+00  5.77348882e-01  1.35364807e+01
  1.70000000e+01  1.00000000e+01  4.34311025e+00  1.69668607e-01
  2.19029031e+00 -2.44196519e-01]


# BA with Motifs Attached

In [19]:
file_list = ['ba_300_20_house_1', 'ba_300_40_house_2', 'ba_300_60_house_3', 'ba_300_80_house_4', 'ba_300_100_house_5', 'ba_300_20_fan_1', 'ba_300_40_fan_2', 'ba_300_60_fan_3', 'ba_300_80_fan_4', 'ba_300_100_fan_5', 'ba_300_20_clique_1', 'ba_300_40_clique_2', 'ba_300_60_clique_3', 'ba_300_80_clique_4','ba_300_100_clique_5', 'ba_300_100_clique_5', 'ba_300_20_diamond_1', 'ba_300_40_diamond_2', 'ba_300_60_diamond_3', 'ba_300_80_diamond_4', 'ba_300_100_diamond_5', 'ba_300_20_cycle_1', 'ba_300_40_cycle_2', 'ba_300_60_cycle_3', 'ba_300_80_cycle_4','ba_300_20_star_1', 'ba_300_40_star_2', 'ba_300_60_star_3', 'ba_300_80_star_4', 'ba_300_100_star_5', 'ba_300_20_grid_1', 'ba_300_40_grid_2', 'ba_300_60_grid_3', 'ba_300_80_grid_4', 'ba_300_100_grid_5']
graph_path = "../Dataset/Validation/Motifs_Attached/BA/"
for i, graph_name in enumerate(file_list):
    GRAPH, _ = input_graph(graph_path,graph_name)
    print(global_feature(GRAPH))
[4.00000000e+02  1.63100000e+03  2.04385965e-02  1.68766871e+01  8.71256591e+00  1.89257056e+00  1.25991998e-01  6.66750000e+00  1.00000000e+01  5.00000000e+00  1.69257043e+01  9.07715843e+00  1.57792467e+01 -4.98249978e-02]
[5.00000000e+02 1.77200000e+03 1.42044088e-02 1.51823828e+01 8.08694357e+00 2.04436815e+00 2.22099539e-01 7.08200000e+00 1.00000000e+01 6.00000000e+00 1.68127960e+01 9.48061445e+00 1.52936640e+01 3.26528131e-02]
[6.00000000e+02 1.91300000e+03 1.06455203e-02 1.45172594e+01 8.13404949e+00 2.19469597e+00 2.95493677e-01 7.42500000e+00 1.00000000e+01 6.00000000e+00 1.70888186e+01 9.31027559e+00 1.54282076e+01 7.98025591e-02]
[7.00000000e+02 2.05500000e+03 8.39975475e-03 1.33271665e+01 7.45022593e+00 2.32776871e+00 3.49301981e-01 7.44285714e+00 1.00000000e+01 5.00000000e+00 1.65173521e+01 8.69846801e+00 1.43816441e+01 1.42201172e-01]
[8.00000000e+02 2.19600000e+03 6.87108886e-03 1.21799544e+01 6.68531876e+00 2.42989583e+00 4.01342087e-01 7.88375000e+00 1.00000000e+01 6.00000000e+00 1.63827030e+01 9.07849329e+00 1.39945147e+01 2.06946443e-01]
[ 3.20000000e+02  1.50900000e+03  2.95650470e-02  1.65505469e+01  7.11414430e+00  1.50145460e+00  0.00000000e+00  4.35000000e+00  6.00000000e+00  3.00000000e+00  1.64147444e+01  9.01327024e+00  1.53652364e+01 -5.48104023e-02]
[ 3.40000000e+02  1.53000000e+03  2.65486726e-02  1.71098398e+01  8.10457516e+00  1.53268247e+00  0.00000000e+00  4.51176471e+00  6.00000000e+00  4.00000000e+00  1.67086257e+01  8.82774475e+00  1.57013009e+01 -7.64124880e-02]
[ 3.60000000e+02  1.55000000e+03  2.39863819e-02  1.56511778e+01  7.03534050e+00  1.55677128e+00  1.11022302e-16  4.83888889e+00  6.00000000e+00  4.00000000e+00  1.59649235e+01  8.37227247e+00  1.45465450e+01 -2.36422190e-02]
[ 3.80000000e+02  1.57000000e+03  2.18025274e-02  1.73542529e+01  9.08588669e+00  1.58720073e+00  3.80279119e-03  4.57894737e+00  6.00000000e+00  3.00000000e+00  1.72521455e+01  9.67105289e+00  1.63240367e+01 -4.85068236e-02]
[ 4.00000000e+02  1.59000000e+03  1.99248120e-02  1.59738912e+01  8.01918239e+00  1.61457243e+00  1.25469720e-03  4.83500000e+00  6.00000000e+00  4.00000000e+00  1.61305661e+01  8.61157732e+00  1.46329416e+01 -3.16377970e-02]
[ 3.01000000e+02  1.50900000e+03  3.34219269e-02  1.81139156e+01  8.06155977e+00  3.01103627e+00  0.00000000e+00  3.67774086e+00  4.00000000e+00  3.00000000e+00  1.71853675e+01  9.60162570e+00  1.65650251e+01 -1.09455409e-01]
[ 3.01000000e+02  1.53000000e+03  3.38870432e-02  1.72193126e+01  7.03323345e+00  2.90235903e+00  0.00000000e+00  3.73754153e+00  4.00000000e+00  3.00000000e+00  1.70450236e+01  9.65734147e+00  1.63625693e+01 -5.62000120e-02]
[ 3.01000000e+02  1.55000000e+03  3.43300111e-02  1.89470113e+01  8.62551281e+00  2.91036712e+00  1.11022302e-16  3.65116279e+00  4.00000000e+00  3.00000000e+00  1.76710816e+01  9.47006322e+00  1.72656096e+01 -1.26352032e-01]
[ 3.01000000e+02  1.57000000e+03  3.47729790e-02  1.88825782e+01  8.42606810e+00  2.87024677e+00  0.00000000e+00  3.62126246e+00  4.00000000e+00  3.00000000e+00  1.77873459e+01  9.24491130e+00  1.74333437e+01 -1.15628568e-01]
[ 3.01000000e+02  1.59000000e+03  3.52159468e-02  1.99480315e+01  9.35974425e+00  2.81350785e+00  0.00000000e+00  3.53820598e+00  4.00000000e+00  3.00000000e+00  1.84072811e+01  9.71809447e+00  1.83275079e+01 -1.14032192e-01]
[ 3.01000000e+02  1.59000000e+03  3.52159468e-02  1.99480315e+01  9.35974425e+00  2.81350785e+00  0.00000000e+00  3.53820598e+00  4.00000000e+00  3.00000000e+00  1.84072811e+01  9.71809447e+00  1.83275079e+01 -1.14032192e-01]
[4.20000000e+02 1.75200000e+03 1.99113536e-02 1.51354286e+01 6.77985975e+00 2.21127964e+00 2.52124285e-01 6.62380952e+00 9.00000000e+00 5.00000000e+00 1.67481574e+01 9.15295321e+00 1.54526017e+01 3.42705124e-02]
[5.40000000e+02 2.01400000e+03 1.38390710e-02 1.36013917e+01 6.13259774e+00 3.30092385e+00 4.13675276e-01 7.21666667e+00 1.00000000e+01 6.00000000e+00 1.63947716e+01 8.58711238e+00 1.45800700e+01 9.40079453e-02]
[6.60000000e+02 2.27700000e+03 1.04704097e-02 1.29828571e+01 6.07496706e+00 3.74403754e+00 5.14613686e-01 7.46818182e+00 1.00000000e+01 6.00000000e+00 1.68873942e+01 9.14159621e+00 1.50003036e+01 1.36415841e-01]
[7.80000000e+02 2.54000000e+03 8.36048846e-03 1.23778566e+01 5.85843933e+00 4.13291798e+00 5.97531930e-01 7.51666667e+00 1.00000000e+01 6.00000000e+00 1.71466297e+01 9.45916821e+00 1.51327173e+01 1.44980734e-01]
[9.00000000e+02 2.80200000e+03 6.92621431e-03 1.11289286e+01 4.89717345e+00 4.46160252e+00 6.54201891e-01 7.91111111e+00 1.00000000e+01 6.00000000e+00 1.63216663e+01 8.34595288e+00 1.37372456e+01 2.08061577e-01]
[ 3.40000000e+02  1.53700000e+03  2.66701371e-02  1.70663846e+01  8.01998163e+00  1.52150584e+00  1.79874524e-02  5.23235294e+00  7.00000000e+00  4.00000000e+00  1.66816411e+01  8.75733363e+00  1.56624465e+01 -7.57489197e-02]
[ 3.79000000e+02  1.58600000e+03  2.21412517e-02  1.62563860e+01  7.88218315e+00  1.55903868e+00  3.73710541e-02  5.49868074e+00  8.00000000e+00  5.00000000e+00  1.65588092e+01  9.07485610e+00  1.53281120e+01 -2.84446946e-02]
[ 4.19000000e+02  1.64300000e+03  1.87619189e-02  1.69293760e+01  9.08204620e+00  1.59348607e+00  5.02945231e-02  5.66825776e+00  8.00000000e+00  5.00000000e+00  1.70018394e+01  9.10292887e+00  1.58219039e+01 -4.02629590e-02]
[4.60000000e+02 1.68300000e+03 1.59420290e-02 1.56240713e+01 8.30233537e+00 1.63352164e+00 7.22403512e-02 5.76739130e+00 8.00000000e+00 4.00000000e+00 1.65938943e+01 9.13459723e+00 1.50985585e+01 1.50458481e-02]
[ 3.20000000e+02  1.50900000e+03  2.95650470e-02  1.58863109e+01  6.45012840e+00  1.50062671e+00  0.00000000e+00  4.50625000e+00  6.00000000e+00  4.00000000e+00  1.62201291e+01  8.77793075e+00  1.50846927e+01 -2.42106824e-02]
[ 3.40000000e+02  1.53000000e+03  2.65486726e-02  1.73157895e+01  8.31045752e+00  1.53360766e+00  0.00000000e+00  4.29705882e+00  6.00000000e+00  3.00000000e+00  1.66841637e+01  8.87666187e+00  1.56660613e+01 -9.45207333e-02]
[ 3.60000000e+02  1.55000000e+03  2.39863819e-02  1.65792191e+01  7.96308244e+00  1.56056692e+00  1.11022302e-16  4.56388889e+00  6.00000000e+00  4.00000000e+00  1.65848006e+01  8.88756965e+00  1.54400607e+01 -4.22406868e-02]
[ 3.80000000e+02  1.57000000e+03  2.18025274e-02  1.67113730e+01  8.44321153e+00  1.58785983e+00  0.00000000e+00  4.63157895e+00  6.00000000e+00  4.00000000e+00  1.66933575e+01  8.63018434e+00  1.55183915e+01 -4.96306344e-02]
[ 4.00000000e+02  1.59000000e+03  1.99248120e-02  1.59707455e+01  8.01603774e+00  1.61380447e+00  0.00000000e+00  4.87000000e+00  6.00000000e+00  4.00000000e+00  1.64697727e+01  9.06985039e+00  1.51217662e+01 -1.00821405e-02]
[4.80000000e+02 1.75200000e+03 1.52400835e-02 1.46550543e+01 7.34783105e+00 1.99046629e+00 2.47539993e-01 9.10000000e+00 1.30000000e+01 7.00000000e+00 1.64382773e+01 8.96145889e+00 1.48127864e+01 6.67076821e-02]
[6.60000000e+02 2.01400000e+03 9.26104750e-03 1.30968703e+01 6.98833017e+00 2.21285509e+00 3.95353799e-01 9.91969697e+00 1.40000000e+01 8.00000000e+00 1.61598788e+01 8.68185077e+00 1.39513859e+01 1.37070806e-01]
[8.40000000e+02 2.27700000e+03 6.46177422e-03 1.21942004e+01 6.76829475e+00 2.39553424e+00 5.12348669e-01 1.03047619e+01 1.40000000e+01 8.00000000e+00 1.64142427e+01 8.96903305e+00 1.39696439e+01 1.89090097e-01]
[1.02000000e+03 2.54000000e+03 4.88752910e-03 1.17755022e+01 6.79126139e+00 2.54651119e+00 5.78671802e-01 1.02647059e+01 1.40000000e+01 7.00000000e+00 1.66424206e+01 8.59273179e+00 1.40183829e+01 1.60527796e-01]
[1.20000000e+03 2.80200000e+03 3.89491243e-03 1.09593002e+01 6.28610278e+00 2.66443972e+00 6.36584211e-01 1.07666667e+01 1.40000000e+01 8.00000000e+00 1.67332203e+01 8.58258582e+00 1.39146225e+01 2.02634647e-01]

/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


[ 4.00000000e+02  1.63100000e+03  2.04385965e-02  1.68766871e+01
  8.71256591e+00  1.89257056e+00  1.25991998e-01  6.66750000e+00
  1.00000000e+01  5.00000000e+00  1.69257043e+01  9.07715843e+00
  1.57792467e+01 -4.98249978e-02]
[5.00000000e+02 1.77200000e+03 1.42044088e-02 1.51823828e+01
 8.08694357e+00 2.04436815e+00 2.22099539e-01 7.08200000e+00
 1.00000000e+01 6.00000000e+00 1.68127960e+01 9.48061445e+00
 1.52936640e+01 3.26528131e-02]
[6.00000000e+02 1.91300000e+03 1.06455203e-02 1.45172594e+01
 8.13404949e+00 2.19469597e+00 2.95493677e-01 7.42500000e+00
 1.00000000e+01 6.00000000e+00 1.70888186e+01 9.31027559e+00
 1.54282076e+01 7.98025591e-02]
[7.00000000e+02 2.05500000e+03 8.39975475e-03 1.33271665e+01
 7.45022593e+00 2.32776871e+00 3.49301981e-01 7.44285714e+00
 1.00000000e+01 5.00000000e+00 1.65173521e+01 8.69846801e+00
 1.43816441e+01 1.42201172e-01]
[8.00000000e+02 2.19600000e+03 6.87108886e-03 1.21799544e+01
 6.68531876e+00 2.42989583e+00 4.01342087e-01 7.88375000e+00
 1.0

# Tree with Motifs Attached

In [18]:
file_list = ['tree_8_20_house_1', 'tree_8_40_house_2', 'tree_8_60_house_3', 'tree_8_80_house_4', 'tree_8_100_house_5', 'tree_8_20_fan_1', 'tree_8_40_fan_2', 'tree_8_60_fan_3', 'tree_8_80_fan_4', 'tree_8_100_fan_5', 'tree_8_20_clique_1', 'tree_8_40_clique_2', 'tree_8_60_clique_3', 'tree_8_80_clique_4', 'tree_8_100_clique_5', 'tree_8_20_diamond_1', 'tree_8_40_diamond_2', 'tree_8_60_diamond_3', 'tree_8_80_diamond_4', 'tree_8_100_diamond_5', 'tree_8_20_cycle_1', 'tree_8_40_cycle_2', 'tree_8_60_cycle_3', 'tree_8_80_cycle_4', 'tree_8_100_cycle_5', 'tree_8_20_star_1', 'tree_8_40_star_2', 'tree_8_60_star_3', 'tree_8_80_star_4', 'tree_8_100_star_5', 'tree_8_20_grid_1', 'tree_8_40_grid_2', 'tree_8_60_grid_3','tree_8_80_grid_4', 'tree_8_100_grid_5']
graph_path ="../Dataset/Validation/Motifs_Attached/Tree/" 
for i, graph_name in enumerate(file_list):
    GRAPH, _ = input_graph(graph_path,graph_name)
    print(global_feature(GRAPH))
[ 6.11000000e+02  7.15000000e+02  3.83676317e-03  2.86554622e+00  5.23910132e-01  2.39334009e+00  6.15864834e-01  1.58248773e+01  1.90000000e+01  1.10000000e+01  3.13717291e+00  1.96699064e-02  1.49381166e+00 -1.36241551e-01]
[ 7.11000000e+02  8.69000000e+02  3.44287950e-03  2.95737327e+00  5.11827132e-01  2.29583498e+00  6.14768241e-01  1.67651195e+01  2.10000000e+01  1.10000000e+01  3.32063587e+00  1.40361693e-01  1.57129500e+00 -1.62792641e-01]
[ 8.11000000e+02  1.02300000e+03  3.11458191e-03  3.01761252e+00  4.93806447e-01  2.23288931e+00  6.27361497e-01  1.66683107e+01  2.10000000e+01  1.10000000e+01  3.32968703e+00  1.44002672e-01  1.62799570e+00 -1.67803340e-01]
[ 9.11000000e+02  1.17700000e+03  2.83953149e-03  3.03911565e+00  4.54259140e-01  2.18280004e+00  6.25118654e-01  1.72425906e+01  2.20000000e+01  1.10000000e+01  3.36282660e+00  5.87622370e-02  1.66797150e+00 -1.37016900e-01]
[ 1.01100000e+03  1.33100000e+03  2.60696693e-03  3.07969925e+00  4.45851457e-01  2.14949380e+00  6.28483016e-01  1.71819980e+01  2.20000000e+01  1.10000000e+01  3.33636289e+00  7.48231412e-02  1.70107888e+00 -1.76173984e-01]
[ 5.31000000e+02  5.83000000e+02  4.14312618e-03  2.76137339e+00  5.64005905e-01  2.56467559e+00  6.12666194e-01  1.48399247e+01  1.80000000e+01  9.00000000e+00  3.03925102e+00  9.81006438e-02  1.37922140e+00 -2.50855699e-01]
[ 5.51000000e+02  6.05000000e+02  3.99274047e-03  2.80479735e+00  6.07298526e-01  2.58162387e+00  6.21353733e-01  1.42323049e+01  1.70000000e+01  9.00000000e+00  3.14082606e+00  8.53085315e-02  1.38653185e+00 -1.74211361e-01]
[ 5.71000000e+02  6.27000000e+02  3.85288967e-03  2.79569034e+00  5.98111263e-01  2.57092316e+00  6.32876231e-01  1.50945709e+01  1.80000000e+01  9.00000000e+00  3.16213833e+00  1.79184507e-01  1.38515906e+00 -2.33888766e-01]
[ 5.91000000e+02  6.49000000e+02  3.72250423e-03  2.81804163e+00  6.20363490e-01  2.58064900e+00  6.04222212e-01  1.52961083e+01  1.80000000e+01  9.00000000e+00  3.02479204e+00  9.30460120e-02  1.38770733e+00 -2.60857636e-01]
[ 6.11000000e+02  6.71000000e+02  3.60065466e-03  2.83296048e+00  6.35195290e-01  2.58679889e+00  6.16842980e-01  1.50654664e+01  1.80000000e+01  9.00000000e+00  3.07985555e+00  9.70343588e-02  1.39028725e+00 -2.39951747e-01]
[ 5.12000000e+02  5.83000000e+02  4.45664139e-03  3.08755365e+00  8.08419543e-01  2.49354091e+00  5.88197912e-01  1.32988281e+01  1.60000000e+01  8.00000000e+00  4.75750785e+00  1.71425015e+00  1.49312387e+00 -1.00716515e-02]
[ 5.12000000e+02  6.05000000e+02  4.62481654e-03  4.10669975e+00  1.74085098e+00  2.45817485e+00  5.84226487e-01  1.22910156e+01  1.60000000e+01  8.00000000e+00  6.53569657e+00  3.44113760e+00  1.98354161e+00 -1.78381563e-02]
[ 5.12000000e+02  6.27000000e+02  4.79299168e-03  5.66241022e+00  3.20947344e+00  2.41222029e+00  5.55219788e-01  1.01601562e+01  1.40000000e+01  7.00000000e+00  7.92236453e+00  4.84820804e+00  3.00074704e+00 -3.70082478e-02]
[ 5.12000000e+02  6.49000000e+02  4.96116683e-03  7.70778720e+00  5.16746316e+00  2.36392366e+00  5.33026987e-01  9.80664062e+00  1.30000000e+01  7.00000000e+00  9.11905104e+00  6.14040853e+00  4.35548764e+00 -5.90099006e-02]
[ 5.12000000e+02  6.71000000e+02  5.12934198e-03  1.02348993e+01  7.60692413e+00  2.32226955e+00  4.95463541e-01  7.45117188e+00  1.00000000e+01  5.00000000e+00  1.01875396e+01  7.05113944e+00  5.77118447e+00 -7.55066773e-02]
[6.31000000e+02 8.47000000e+02 4.26131361e-03 3.35147929e+00 6.68383050e-01 2.20098686e+00 6.88149995e-01 1.56798732e+01 2.00000000e+01 1.10000000e+01 4.35369129e+00 5.29604404e-02 2.07954711e+00 2.59788067e-01]
[7.51000000e+02 1.13300000e+03 4.02308034e-03 3.68435013e+00 6.67597073e-01 2.04599054e+00 7.27342059e-01 1.57283622e+01 2.00000000e+01 1.10000000e+01 4.43326177e+00 5.98464656e-02 2.41861190e+00 2.44719953e-01]
[8.71000000e+02 1.41900000e+03 3.74519973e-03 3.86591390e+00 6.07779124e-01 1.95186782e+00 7.45606167e-01 1.61814007e+01 2.00000000e+01 1.10000000e+01 4.52163055e+00 1.20515826e-01 2.62655247e+00 2.84630880e-01]
[9.91000000e+02 1.70500000e+03 3.47572598e-03 4.03229595e+00 5.91289346e-01 1.90083610e+00 7.65651482e-01 1.48990918e+01 1.90000000e+01 1.10000000e+01 4.58019205e+00 2.09155890e-02 2.77043627e+00 2.37600066e-01]
[1.11100000e+03 1.99100000e+03 3.22897155e-03 4.10507793e+00 5.20813960e-01 1.85553657e+00 7.79083601e-01 1.51584158e+01 1.90000000e+01 1.10000000e+01 4.57599163e+00 3.95854930e-02 2.86805134e+00 2.39052815e-01]
[ 6.31000000e+02  6.66000000e+02  3.35069051e-03  2.54736842e+00  4.35611523e-01  2.59889825e+00  6.40153667e-01  1.92266244e+01  2.40000000e+01  1.20000000e+01  2.81830971e+00  7.62507451e-02  1.31321256e+00 -2.49852659e-01]
[ 7.51000000e+02  8.17000000e+02  2.90102086e-03  2.56985294e+00  3.93389801e-01  2.48808356e+00  6.20719592e-01  1.95259654e+01  2.40000000e+01  1.20000000e+01  2.90294576e+00  6.02127736e-02  1.34299836e+00 -1.72090207e-01]
[ 8.71000000e+02  9.69000000e+02  2.55750426e-03  2.58574380e+00  3.60110616e-01  2.41646207e+00  5.96702099e-01  1.98140069e+01  2.40000000e+01  1.20000000e+01  2.90010860e+00  5.04971390e-02  1.36522252e+00 -1.26983061e-01]
[ 9.91000000e+02  1.11200000e+03  2.26686644e-03  2.56705671e+00  3.22348983e-01  2.37737050e+00  5.92770253e-01  1.99162462e+01  2.40000000e+01  1.20000000e+01  2.90950210e+00  3.59844916e-02  1.36974833e+00 -9.30059849e-02]
[ 1.11100000e+03  1.26900000e+03  2.05804364e-03  2.58123028e+00  2.96343819e-01  2.32676138e+00  5.77732571e-01  2.00738074e+01  2.40000000e+01  1.20000000e+01  2.93749250e+00  4.89112638e-02  1.38767332e+00 -5.70437293e-02]
[ 5.31000000e+02  5.83000000e+02  4.14312618e-03  2.76480687e+00  5.67436437e-01  2.56370465e+00  6.17586918e-01  1.49359699e+01  1.80000000e+01  9.00000000e+00  3.05620799e+00  1.35058180e-01  1.38014119e+00 -2.22806655e-01]
[ 5.51000000e+02  6.05000000e+02  3.99274047e-03  2.80148883e+00  6.03992740e-01  2.57715209e+00  6.21275869e-01  1.51343013e+01  1.80000000e+01  9.00000000e+00  3.10805763e+00  1.35779157e-01  1.38697963e+00 -2.00324412e-01]
[ 5.71000000e+02  6.27000000e+02  3.85288967e-03  2.80526736e+00  6.07680641e-01  2.57548879e+00  6.20202275e-01  1.53012259e+01  1.80000000e+01  9.00000000e+00  3.02101061e+00  9.77187240e-02  1.38609480e+00 -2.58761236e-01]
[ 5.91000000e+02  6.49000000e+02  3.72250423e-03  2.80724749e+00  6.09577666e-01  2.57690433e+00  6.26616509e-01  1.55465313e+01  1.80000000e+01  9.00000000e+00  3.17214071e+00  7.40320354e-02  1.39089755e+00 -2.09390112e-01]
[ 6.11000000e+02  6.71000000e+02  3.60065466e-03  2.82252051e+00  6.24763099e-01  2.58173187e+00  6.02109759e-01  1.54173486e+01  1.80000000e+01  9.00000000e+00  3.05295496e+00  1.02951782e-01  1.38839719e+00 -2.72250725e-01]
[1.23100000e+03 1.90300000e+03 2.51365471e-03 3.52839117e+00 4.35792731e-01 1.96977031e+00 5.13850548e-01 1.49512591e+01 1.80000000e+01 1.20000000e+01 3.99096859e+00 3.75323476e-02 1.93406894e+00 2.44342399e-01]
[1.95100000e+03 3.24500000e+03 1.70589704e-03 3.68988903e+00 3.62869028e-01 1.88797249e+00 4.82335607e-01 1.54787289e+01 1.90000000e+01 1.20000000e+01 4.00903026e+00 7.93874351e-02 2.05247189e+00 2.51233596e-01]
[2.67100000e+03 4.58700000e+03 1.28639276e-03 3.74640209e+00 3.11352701e-01 1.85275206e+00 4.73319804e-01 1.61463871e+01 2.10000000e+01 1.30000000e+01 4.01596445e+00 7.75234578e-02 2.10318175e+00 2.35983456e-01]
[3.39100000e+03 5.92900000e+03 1.03153498e-03 3.79352227e+00 2.96316199e-01 1.83533189e+00 4.86666758e-01 1.68596284e+01 2.30000000e+01 1.30000000e+01 4.08792989e+00 7.97548799e-02 2.13446061e+00 2.54086378e-01]
[4.11100000e+03 7.27100000e+03 8.60666386e-04 3.80990371e+00 2.72315946e-01 1.82294486e+00 4.77064106e-01 1.62816833e+01 2.10000000e+01 1.40000000e+01 4.10287047e+00 9.62138460e-02 2.15217682e+00 2.64461514e-01]

/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
/tmp/ipykernel_914560/649938460.py:21: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


[ 6.11000000e+02  7.15000000e+02  3.83676317e-03  2.86554622e+00
  5.23910132e-01  2.39334009e+00  6.15864834e-01  1.58248773e+01
  1.90000000e+01  1.10000000e+01  3.13717291e+00  1.96699064e-02
  1.49381166e+00 -1.36241551e-01]
[ 7.11000000e+02  8.69000000e+02  3.44287950e-03  2.95737327e+00
  5.11827132e-01  2.29583498e+00  6.14768241e-01  1.67651195e+01
  2.10000000e+01  1.10000000e+01  3.32063587e+00  1.40361693e-01
  1.57129500e+00 -1.62792641e-01]
[ 8.11000000e+02  1.02300000e+03  3.11458191e-03  3.01761252e+00
  4.93806447e-01  2.23288931e+00  6.27361497e-01  1.66683107e+01
  2.10000000e+01  1.10000000e+01  3.32968703e+00  1.44002672e-01
  1.62799570e+00 -1.67803340e-01]
[ 9.11000000e+02  1.17700000e+03  2.83953149e-03  3.03911565e+00
  4.54259140e-01  2.18280004e+00  6.25118654e-01  1.72425906e+01
  2.20000000e+01  1.10000000e+01  3.36282660e+00  5.87622370e-02
  1.66797150e+00 -1.37016900e-01]
[ 1.01100000e+03  1.33100000e+03  2.60696693e-03  3.07969925e+00
  4.45851457e-01  2

'[ 6.11000000e+02  7.15000000e+02  3.83676317e-03  2.86554622e+00  5.23910132e-01  2.39334009e+00  6.15864834e-01  1.58248773e+01  1.90000000e+01  1.10000000e+01  3.13717291e+00  1.96699064e-02  1.49381166e+00 -1.36241551e-01]\n[ 7.11000000e+02  8.69000000e+02  3.44287950e-03  2.95737327e+00  5.11827132e-01  2.29583498e+00  6.14768241e-01  1.67651195e+01  2.10000000e+01  1.10000000e+01  3.32063587e+00  1.40361693e-01  1.57129500e+00 -1.62792641e-01]\n[ 8.11000000e+02  1.02300000e+03  3.11458191e-03  3.01761252e+00  4.93806447e-01  2.23288931e+00  6.27361497e-01  1.66683107e+01  2.10000000e+01  1.10000000e+01  3.32968703e+00  1.44002672e-01  1.62799570e+00 -1.67803340e-01]\n[ 9.11000000e+02  1.17700000e+03  2.83953149e-03  3.03911565e+00  4.54259140e-01  2.18280004e+00  6.25118654e-01  1.72425906e+01  2.20000000e+01  1.10000000e+01  3.36282660e+00  5.87622370e-02  1.66797150e+00 -1.37016900e-01]\n[ 1.01100000e+03  1.33100000e+03  2.60696693e-03  3.07969925e+00  4.45851457e-01  2.1494938

# Real World Network

In [25]:
file_list = ["test","testTrain","corruption","foodweb-baywet","inf-USAir97","moreno_crime_projected",'opsahl-openflights','household','faa','facebook','powergrid','netscience','HI-II-14']
graph_path =  "../Dataset/Real/"
for i, graph_name in enumerate(file_list):
    GRAPH, _ = input_graph(graph_path,graph_name)
    print(global_feature(GRAPH))
[17.         17.          0.125       3.3030303   1.23529412  3.21415762  0.44809689  4.29411765  5.          3.          2.92069242  0.89199137  1.46892545 -0.45761079]
[ 5.00000000e+01  1.84000000e+02  1.50204082e-01  1.03516484e+01  2.92260870e+00  3.13906005e+00  0.00000000e+00  2.98000000e+00  4.00000000e+00  2.00000000e+00  9.78139874e+00  5.71596089e+00  8.52071314e+00 -2.10216254e-01]
[3.09000000e+02 3.28100000e+03 6.89488505e-02 3.49304984e+01 1.36890817e+01 1.37170794e+00 6.75389581e-01 5.28155340e+00 7.00000000e+00 4.00000000e+00 4.65126424e+01 9.92453240e+00 5.88368426e+01 5.32419469e-01]
[ 1.28000000e+02  2.07500000e+03  2.55290354e-01  4.02034884e+01  7.68511295e+00  1.37534330e+00  0.00000000e+00  2.85937500e+00  3.00000000e+00  2.00000000e+00  3.96313826e+01  2.56238824e+01  5.01873144e+01 -1.11663002e-01]
[ 3.32000000e+02  2.12600000e+03  3.86925345e-02  4.43728534e+01  3.15554240e+01  1.58372427e+00  7.24340668e-02  5.09337349e+00  6.00000000e+00  3.00000000e+00  4.12334160e+01  2.39248865e+01  5.11165573e+01 -2.07876231e-01]
[7.54000000e+02 2.12700000e+03 7.49257611e-03 1.11250882e+01 5.48079823e+00 1.72960174e+00 8.00633227e-01 1.23023873e+01 1.60000000e+01 8.00000000e+00 1.80658069e+01 4.00108109e+00 1.65582780e+01 1.41836763e-01]
[2.93900000e+03 1.56770000e+04 3.63112815e-03 5.57336459e+01 4.50636457e+01 1.72268983e+00 5.94581336e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00 6.29987341e+01 2.06317455e+01 7.93673426e+01 5.09155499e-02]
[ 9.21000000e+02  4.03200000e+03  9.51706557e-03  3.39443135e+01  2.51845278e+01  1.70777367e+00  4.20284180e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00  3.19726603e+01  1.65007810e+01  3.62813085e+01 -1.20689026e-01]
[ 1.22600000e+03  2.40800000e+03  3.20671172e-03  7.35895305e+00  3.42850572e+00  1.90541897e+00  4.58476417e-01  1.26598695e+01  1.70000000e+01  9.00000000e+00  9.20927318e+00  1.37955937e+00  4.29571519e+00 -1.52027501e-02]
[ 2.88800000e+03  2.98100000e+03  7.15069079e-04  2.55870324e+02  2.53763170e+02  2.55893091e+01  7.64527403e-01  6.75969529e+00  9.00000000e+00  5.00000000e+00  2.78030792e+01  1.15649212e+00  2.90139187e+01 -6.68214007e-01]
[4.94100000e+03 6.59400000e+03 5.40302697e-04 3.87146432e+00 1.20215126e+00 2.24677791e+00 5.87513807e-01 3.45407812e+01 4.60000000e+01 2.30000000e+01 7.48305133e+00 8.73806296e-01 2.11145090e+00 3.45698774e-03]
[1.46100000e+03 2.74200000e+03 2.57095440e-03 6.93981397e+00 3.18513742e+00 1.97078376e+00 9.04238895e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.90238140e+01 8.64835535e+00 1.69346215e+01 4.61622467e-01]
[ 4.16500000e+03  1.30870000e+04  1.50919157e-03  3.99966760e+01  3.37109124e+01  1.96434765e+00  1.44344081e-01  7.75582233e+00  1.10000000e+01  6.00000000e+00  3.49332947e+01  1.02577100e+01  3.83743873e+01 -2.01580171e-01]

/tmp/ipykernel_914560/4291687802.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
/tmp/ipykernel_914560/4291687802.py:21: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


[17.         17.          0.125       3.3030303   1.23529412  3.21415762
  0.44809689  4.29411765  5.          3.          2.92069242  0.89199137
  1.46892545 -0.45761079]
[ 5.00000000e+01  1.84000000e+02  1.50204082e-01  1.03516484e+01
  2.92260870e+00  3.13906005e+00  0.00000000e+00  2.98000000e+00
  4.00000000e+00  2.00000000e+00  9.78139874e+00  5.71596089e+00
  8.52071314e+00 -2.10216254e-01]
[3.09000000e+02 3.28100000e+03 6.89488505e-02 3.49304984e+01
 1.36890817e+01 1.37170794e+00 6.75389581e-01 5.28155340e+00
 7.00000000e+00 4.00000000e+00 4.65126424e+01 9.92453240e+00
 5.88368426e+01 5.32419469e-01]
[ 1.28000000e+02  2.07500000e+03  2.55290354e-01  4.02034884e+01
  7.68511295e+00  1.37534330e+00  0.00000000e+00  2.85937500e+00
  3.00000000e+00  2.00000000e+00  3.96313826e+01  2.56238824e+01
  5.01873144e+01 -1.11663002e-01]
[ 3.32000000e+02  2.12600000e+03  3.86925345e-02  4.43728534e+01
  3.15554240e+01  1.58372427e+00  7.24340668e-02  5.09337349e+00
  6.00000000e+00  3.00000

# Cancer Gene Network

In [26]:
graph_path = "../Dataset/GeneNetwork/"
file_list =  ["COAD","HNSC","KICH","PRAD","UCEC"]
for i, graph_name in enumerate(file_list):
    GRAPH, _ = input_graph(graph_path,graph_name)
    print(global_feature(GRAPH))
[ 4.38700000e+03  1.62760000e+04  1.69177037e-03  3.33048754e+01  2.58837782e+01  1.83044763e+00  6.79641060e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00  3.80132156e+01  1.24791194e+00  4.31148035e+01 -1.52539303e-01]
[5.39300000e+03 4.04470000e+04 2.78186472e-03 9.95358189e+01 8.45347863e+01 1.67404421e+00 5.99583730e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.62055371e+02 1.25115408e+02 2.21399876e+02 5.22821890e-01]
[ 2.61800000e+03  5.26900000e+03  1.53810091e-03  3.42432381e+01  3.02148734e+01  2.30281030e+00  6.83436290e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.99856383e+01  1.46361339e+01  3.19064343e+01 -2.25141127e-01]
[4.57400000e+03 2.04650000e+04 1.95679073e-03 5.26425029e+01 4.36928371e+01 1.81494545e+00 6.49035745e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00 9.00711838e+01 5.87793702e+01 1.17786325e+02 3.15917940e-01]

/tmp/ipykernel_914560/4291687802.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')
/tmp/ipykernel_914560/4291687802.py:21: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  A = nx.to_scipy_sparse_matrix(g, weight='weight',dtype=np.float,format='csr')


[ 4.38700000e+03  1.62760000e+04  1.69177037e-03  3.33048754e+01
  2.58837782e+01  1.83044763e+00  6.79641060e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.80132156e+01  1.24791194e+00
  4.31148035e+01 -1.52539303e-01]
[5.39300000e+03 4.04470000e+04 2.78186472e-03 9.95358189e+01
 8.45347863e+01 1.67404421e+00 5.99583730e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.62055371e+02 1.25115408e+02
 2.21399876e+02 5.22821890e-01]
[ 2.61800000e+03  5.26900000e+03  1.53810091e-03  3.42432381e+01
  3.02148734e+01  2.30281030e+00  6.83436290e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.99856383e+01  1.46361339e+01
  3.19064343e+01 -2.25141127e-01]
[4.57400000e+03 2.04650000e+04 1.95679073e-03 5.26425029e+01
 4.36928371e+01 1.81494545e+00 6.49035745e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.00711838e+01 5.87793702e+01
 1.17786325e+02 3.15917940e-01]
